In [1]:


# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.add("Distributions")
# Pkg.add("CxxWrap")
# Pkg.add("JLD")
# Pkg.add("ImageIO")
# Pkg.add("CircularArrays")
# Pkg.add("OffsetArrays")
# Pkg.add("TensorOperations")
# Pkg.add("TensorCast")
# Pkg.add("Tullio")
# Pkg.add("DiffEqOperators")
# Pkg.add("BandedMatrices")
# Pkg.add("PyPlot")
# Pkg.add("Plots")
# Pkg.add("FFTW")
# Pkg.add("SciPy")
# Pkg.add("GR")
# Pkg.add("Statistics")

import GR
using BenchmarkTools
using Distributions
using Random
using Plots
using PyCall
using PyPlot
#using Images
using CircularArrays
using CxxWrap
using FFTW
using SciPy
using JLD
using Statistics
stats = pyimport("scipy.stats")
np = pyimport("numpy")
cv2 = pyimport("cv2")



PyObject <module 'cv2' from '/usr/lib/python3/dist-packages/cv2.cpython-38-x86_64-linux-gnu.so'>

In [2]:
function Laplacian_2D!(P₁,P₂,A₁,A₂,Δx)
    B₁ = CircularArrays.CircularArray(A₁);
    B₂ = CircularArrays.CircularArray(A₂);
    Threads.@threads for 😄 ∈ CartesianIndices(P₁)
        (i,j) = Tuple(😄)
        P₁[i,j] = @fastmath (-B₁[i+2,j] + 16B₁[i+1,j] 
            + 16B₁[i-1,j] -B₁[i-2,j] 
            -B₁[i,j+2] + 16B₁[i,j+1] 
            + 16B₁[i,j-1] -B₁[i,j-2] 
            - 60B₁[i,j]) /(12Δx^2);
        P₂[i,j] = @fastmath (-B₂[i+2,j] + 16B₂[i+1,j] 
            + 16B₂[i-1,j] -B₂[i-2,j] 
            -B₂[i,j+2] + 16B₂[i,j+1] 
            + 16B₂[i,j-1] -B₂[i,j-2] 
            - 60B₂[i,j]) /(12Δx^2);  
    end
    return nothing
end



function Cores_2D!(N,angle,thr)
    s = []
    count = 0
    accept = 0.5 - 0.5*thr/100
    for 😄 ∈ 1:(N-1)
        for 🥪 ∈ 1:(N-1)
            norm₁ = (angle[😄,🥪] + π)/(2π)
            norm₂ = (angle[😄+1,🥪] + π)/(2π)
            norm₃ = (angle[😄+1,🥪+1] + π)/(2π)
            norm₄ = (angle[😄,🥪+1] + π)/(2π)

            θ₁ = min(abs(norm₂ - norm₁), 1 - abs(norm₂ - norm₁))
            θ₂ = min(abs(norm₃ - norm₂), 1 - abs(norm₃ - norm₂))
            θ₃ = min(abs(norm₄ - norm₃), 1 - abs(norm₄ - norm₃))
            θₛ = θ₁ + θ₂ + θ₃

            if θₛ > accept 
                append!(s,[[😄,🥪]])
            end
        end
    end

    if length(s) > 0
        for 🇸🇦 ∈ 1:(length(s)-1)
  
            diffᵣ = s[🇸🇦 + 1][1] - s[🇸🇦][1]
            diffₛ = s[🇸🇦 + 1][2] - s[🇸🇦][2]

            if diffᵣ == 0 && diffₛ == 1
                count += 1
            end
            if diffᵣ == 1 && diffₛ == 0
                count += 1
            end
        end
    end

    num = length(s) - count

    return num
end



function setting!(moo)
    @inbounds Threads.@threads for t in eachindex(moo)
        if moo[t] > 1
            moo[t] = 1
        end
    end
    return nothing
end

function mooing!(moo,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(moo)
        (i,j) = Tuple(😄)
        @fastmath moo[i,j] = sqrt(A₁[i,j]^2 + A₂[i,j]^2)
    end
    return nothing
end


function angler!(angle,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(angle)
        (i,j) = Tuple(😄)
        angle[i,j] = @fastmath atan(A₂[i,j],A₁[i,j]);
    end
    return nothing
end

function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end


function minmax(A)
    lower = minimum(minimum.(A))
    upper = maximum(maximum.(A))
    return lower,upper
end

minmax (generic function with 1 method)

In [3]:
function PQfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j]);
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j]);
    end
    return nothing
end

function PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ)

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    PQfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,1,fₐ)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    
    PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,time + Δt,1,fₐ)

    return nothing
end



PQupdate_2D! (generic function with 1 method)

In [4]:
function EQCDfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j])
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j])
    end
    return nothing
end


function EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ,r,s,n)



    #λ = (fₐ/mass(fₐ))^2;

    λᵪ = [1024 1448 3072 3584 5504 1];
    #ηₓ,η = ηtime(time,fₐ);
    λ = λᵪ[r]

    ηᵪ = [2.8 3.0 3.2 3.4 3.6 0 6]
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end
    

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    EQCDfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,λ,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = ηtime(time+Δt,fₐ);

    EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂, time + Δt,λ,ηₓ,n)

    return nothing
end

EQCDupdate_2D! (generic function with 1 method)

In [5]:
function LLaplacian_2D!(P,A,Δx)
    B = CircularArray(A);
    Threads.@threads for 😄 ∈ CartesianIndices(P)
        (i,j) = Tuple(😄)
        P[i,j] = @fastmath (-B[i+2,j] + 16B[i+1,j] 
            + 16B[i-1,j] -B[i-2,j] 
            -B[i,j+2] + 16B[i,j+1] 
            + 16B[i,j-1] -B[i,j-2] 
            - 60B[i,j]) /(12Δx^2);
    end
    return nothing
end


function Lfupdate_2D!(F,M,A,Ȧ,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F)
        (i,j) = Tuple(😄)
        F[i,j] = @fastmath M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j];
    end
    return nothing
end


function LAupdate_2D!(A,Δt,Ȧ,F)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A)
        (i,j) = Tuple(😄)
        A[i,j] = @fastmath A[i,j] .+ Δt .* (Ȧ[i,j] .+ 0.5Δt .* F[i,j])
    end
    return nothing
end


function Lvelupdate_2D!(Ȧ,Δt,F,M,A,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ)
        (i,j) = Tuple(😄)
        Ȧ[i,j] = @fastmath Ȧ[i,j] .+ 0.5Δt .* (F[i,j] .+ M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j])
    end
    return nothing
end


function Lupdate_2D!(A,Ȧ,M,F,Δx,Δt,time,fₐ,s,n)

    #ηₓ,η = Lηtime(time,fₐ);


    ηᵪ = [2.8 3.0 3.2 3.4 3.6 0 6]

    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    Lfupdate_2D!(F,M,A,Ȧ,time,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    LAupdate_2D!(A,Δt,Ȧ,F)

    LLaplacian_2D!(M,A,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = Lηtime(time+Δt,fₐ)

    Lvelupdate_2D!(Ȧ,Δt,F,M,A,time + Δt,ηₓ,n)

    return nothing
end


Lupdate_2D! (generic function with 1 method)

In [6]:
function FPQrun_2D!(n,N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,repeats,cont,low)
    #700 images on per 5000 updates
    
    ∇A = zeros(N,N);
    ∇r = zeros(N,N);
    ρᵣ = zeros(N,N);
    ρₐ = zeros(N,N);
    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);
    ∇A₁ = zeros(N,N);
    ∇A₂ = zeros(N,N);

    moo = zeros(N,N);
    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N);

    # fₛ = Δx
    # freq_s = fₛ/2 * range(-1,stop = 1,length = N)
    # freq = fₛ/2 * range(0,stop = 1, length = N/2)
    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

    x = 500

    tracker = []
    cores = []
    thr = 1
    
    saxion = []
    axion = []
    axionmax = []
    axionmin = []

    axenergy = []
    
    lo = low
    
    tim = t₀
    
    for 🥪 ∈ 1+cont:repeats+cont
        


        while tim < 100*🥪 + t₀
            tim = round(tim,digits = 8);


            if lo % x == 0
                #125 pictures
                mooing!(moo,A₁,A₂);
                #println(moo[1,1])
                #setting!(moo);
                angler!(angle,A₁,A₂);
                
    

                strng = Cores_2D!(N,angle,thr)
                append!(tracker,tim)
                append!(cores,strng)


                ṙ = (A₁ .* Ȧ₁ .+ A₂ .* Ȧ₂)./sqrt.(A₁.^2 .+ A₂.^2)
                Ȧ = (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)


                Laplacian_2D!(∇A,∇r, angle, moo, Δx)

                ρᵣ .= ṙ.^2 + ∇r.^2
                ρₐ .= Ȧ.^2 .+ ∇A.^2 


                # PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                f_image = FFTW.fft(ρᵣ)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                append!(saxion,[Abin])

                f_image = FFTW.fft(ρₐ .* fₐ^2)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


                append!(axion,[Abins])
                
#                 r = sqrt.(A₁.^2 .+ A₂.^2)
                
#                 Ȧ = (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./r.^2
#                 ṙ = (A₁ .* Ȧ₁ .+ A₂ .* Ȧ₂)./r
#                 λᵣ = 1
#                 #Vᵣ = λᵣ./4 .*  r.^2 .* (r .+ 2fₐ)
                
                
#                 Laplacian_2D!(∇A,∇r, angle.*fₐ, r, Δx)
                
                
                
                Laplacian_2D!(∇A₁,∇A₂,A₁,A₂,Δx)
                

                ρ = Ȧ₁.^2 .+ Ȧ₂.^2 .+ ∇A₁.^2 .+ ∇A₂.^2 #.+ 1/4 .* (A₁.^2 .+ A₂.^2 .- 1).^2
                
#                 ρ = 1/2 .* Ȧ.^2 .+ 1/2 .* ∇A.^2 .+ 1/2 .* ṙ.^2 .+ 1/2 .* ∇r.^2

                rho = mean(ρ)
                
                amax = maximum((log.(ρ/rho)))
                amin = minimum((log.(ρ/rho)))

                append!(axionmax,amax)
                append!(axionmin,amin)


                #PyPlot.imsave("PQTOWN/new/O/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin=0,vmax = 4,cmap = "inferno")


                f_image = FFTW.fft( ρ .* fₐ^2)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                append!(axenergy,[Abint])

                
                
                

                
                
                #PyPlot.imsave("PQTOWN/new/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
                
            end
            
            if tim < 250
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ)
                tim = tim + Δt
            elseif tim ≥ 250 && tim < 500
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 250/tim,tim,fₐ)
                tim = tim + Δt * 250/tim
            else
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 0.5 * 500/tim,tim,fₐ)
                tim = tim + Δt * 0.5 * 500/tim
            end
            lo += 1
        end
     


        save("PQTOWN/new/Saving/"*string(i)*"/Box9"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)

        save("PQTOWN/new/Saving/"*string(i)*"/MinMax9"*string(n)*string(🥪)*".jld", "max", axionmax,"min",axionmin)



        save("PQTOWN/new/Saving/"*string(i)*"/Strings9"*string(n)*string(🥪)*".jld","time",tracker,"number",cores,"lo",lo)
        save("PQTOWN/new/Saving/"*string(i)*"/FFT9"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)


        println(tim)
    end



    return nothing
end

FPQrun_2D! (generic function with 1 method)

In [7]:
function FErun_2D!(n,N,t₁,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeats,cont,low,B,aang,poggers,cap,pic)
    
    ∇r = zeros(N,N);
    ∇A = zeros(N,N);
    ∇A₁ = zeros(N,N);
    ∇A₂ = zeros(N,N);
    ∇angle = zeros(N,N);
    moo = zeros(N,N);
    angll = zeros(N,N);
    
    mooing!(moo,A₁,A₂);
            
    angler!(angll,A₁,A₂);
    
    Laplacian_2D!(∇A₁,∇A₂,A₁,A₂,Δx)
    
    
    Laplacian_2D!(∇A,∇r, angll, moo, Δx)
    
    ρₐ = zeros(N,N);
    ρᵣ = zeros(N,N);
    ρ = zeros(N,N);

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)


    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])
    
    x = cap

    tracker = []
    cores = []
    thr = 1
    lo = low
    strng = 1
    mng = 0

    saxion = []
    axion = []
    axenergy = []
    axionmax = []
    axionmin = []
    
    Ȧ = zeros(N,N)

    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

    M = zeros(N,N);

    F = zeros(N,N);

    LLaplacian_2D!(M,angll,Δx)
    
    tim = t₁
    
    ay,az = minmax(angll)
    star = 3

    
    if B != 0
        Ȧ = B
        strng = 0
        mng = 2
        angll = aang
        ay = 0
        az = 0
    end
    
    
    for 🥪 ∈ 1+cont:repeats + cont
        

        while tim < 0.5*🥪 + 0.4
        #while strng != 0
            
            #if az ≥ 1 && ay ≤ -1
            if tim < 10


                tim = round(tim,digits = 10);
                if lo % x == 0
                    #18 pictures
                    mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    angler!(angll,A₁,A₂);
                    
                    

                    strng = Cores_2D!(N,angll,thr)
                    
          
                    
                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                    ṙ = (A₁ .* Ȧ₁ .+ A₂ .* Ȧ₂)./sqrt.(A₁.^2 .+ A₂.^2)
                    Ȧ = (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)
                    
                    
                    Laplacian_2D!(∇A,∇r, angll, moo, Δx)
                    
                    ρᵣ .= ṙ.^2 + ∇r.^2
                    ρₐ .= Ȧ.^2 .+ ∇A.^2 


                    f_image = FFTW.fft(moo)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    append!(saxion,[Abin])

                    f_image = FFTW.fft(angll)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/angll/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                
                    
                    
#                                     r = sqrt.(A₁.^2 .+ A₂.^2)
                
#                 Ȧ = (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./r.^2
#                 ṙ = (A₁ .* Ȧ₁ .+ A₂ .* Ȧ₂)./r
#                 λᵣ = 1
#                 #Vᵣ = λᵣ./4 .*  r.^2 .* (r .+ 2fₐ)
                
                
# #                 
                    
                    
                     Laplacian_2D!(∇A₁,∇A₂,A₁,A₂,Δx)


                    ρ .= Ȧ₁.^2 .+ Ȧ₂.^2 .+ ∇A₁.^2 .+ ∇A₂.^2 #.+ 1/4 .* (A₁.^2 .+ A₂.^2 .- 1).^2

                    
                    rho = mean(ρ)
                    
#                     amax = maximum((log.(ρ/rho)))
#                     amin = minimum((log.(ρ/rho)))
                    
#                     append!(axionmax,amax)
#                     append!(axionmin,amin)
                    
                    PyPlot.imsave("Final/O/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin = 0,vmax = 6,cmap = "inferno")


                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])


                    
                    
                    

                    PyPlot.imsave("Final/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int, pic+ lo/x )) ,3,"0")*".png",angll,vmin=-π,vmax = π,cmap = "twilight")
                    
                    ay,az = minmax(angll)

                end
                if tim < 3
                
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ,r,s,poggers)
                    tim = tim + Δt
                    
                else
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt*(3/tim)^3.34,tim,fₐ,r,s,poggers)
                    tim = tim + Δt*(3/tim)^3.34
                    
                end
                lo += 1
                
                


                
            else
                
                if mng == 0
                    Ȧ = zeros(N,N)

                    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_2D!(M,angll,Δx)
                elseif mng == 2
                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_2D!(M,angll,Δx)
                end


                tim = round(tim,digits = 7);
                if lo % x == 0
                    #13 pictures
                    #mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    #angllr!(angll,A₁,A₂);

                    strng = Cores_2D!(N,angll,thr)

        

                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")



                    f_image = FFTW.fft(angll)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

#                     plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
#                     Plots.savefig(plotc,"EQCD/angll/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])
                    
                    LLaplacian_2D!(∇angle,angll,Δx )
                    
                    
                    ρ .= Ȧ.^2 + ∇angle.^2
                    rho = mean(ρ)
                    
                    
                    amax = maximum((log.(ρ/rho)))
                    amin = minimum((log.(ρ/rho)))
                    
                    append!(axionmax,amax)
                    append!(axionmin,amin)
                    
                    
#                     PyPlot.imsave("Final/O/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",(log.(ρ/rho)),vmin = 0,vmax = 6,cmap = "inferno")


                    f_image = FFTW.fft( ρ .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])





                    

#                     PyPlot.imsave("Final/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/"*lpad( string(trunc(Int,pic + lo/x )) ,3,"0")*".png",angll,vmin=-π,vmax = π,cmap = "twilight")
                    
                    ay,az = minmax(angll)

                end
                
                if tim < 3
                    Lupdate_2D!(angll,Ȧ,M,F,Δx,Δt,tim,fₐ,s,poggers)
                    tim = tim + Δt
                else
                    Lupdate_2D!(angll,Ȧ,M,F,Δx,Δt*(3/tim)^3.34,tim,fₐ,s,poggers)
                    tim = tim + Δt*(3/tim)^3.34
                end
                
                lo += 1
   
                mng = 1

               
            end
           

        end

        save("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂,"angle",angll,"AngVel",Ȧ)
        save("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/MinMax"*string(n)*string(🥪)*".jld", "max", axionmax,"min",axionmin)



        save("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores,"lo",lo)
        save("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)


        println(tim)
 

        
    end


    

    return nothing
end

FErun_2D! (generic function with 1 method)

In [9]:
@time begin

#         λᵪ = [1024 1448 3072 3584 5504 1];
#     #ηₓ,η = ηtime(time,fₐ);
#     λ = λᵪ[r]

#     ηᵪ = [2.8 2.8 3.2 3.4 3.6 0 400]

    
n = 9
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
i = 5
    
eta = [1 5 7 2 3 4 6]
rr = [1 5 2 3 4 6]
pog = [1 2 3 4 5 6 6.68]
    
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]





cont = 0
low = 0
Ȧ = 0
angle = 0
cap = 25
Ȧ = 0
angll = 0
pic = 0#2488
#poggers = 6.68

t₀ = 0.0001;
print(string(i)*" ")


Ȧ₁ = zeros(N,N);
Ȧ₂ = zeros(N,N);


μ,σ = 0, 0.1;
C₁ = rand(Normal(μ,σ),N,N);
C₂ = rand(Normal(μ,σ),N,N);


Δx = round(LPQ[7]/N,digits = 4)
Δt = 0.004#0.000001

     repea = 8



#low = load("PQTOWN/new/Saving/"*string(i)*"/Strings"*string(n)*string(repeat)*".jld")["lo"]



#  FPQrun_2D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repea,cont,low)


            




LQCD = LPQ/(repea*100)*0.4

Δx = round(LQCD[7]/N,digits = 6)
Δt = 0.001


for poggrs ∈ 1:1

for aa ∈ 1:1

for nl ∈ 7:7


                
s = eta[nl]
r = rr[aa]
poggers = pog[poggrs]
                
                
C₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Real"]
C₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Imaginary"]
Ȧ₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Realvel"]
Ȧ₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["RealIm"]

# C₁ = load("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/Box"*string(n)*string(cont)*".jld")["Real"]
# C₂ = load("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/Box"*string(n)*string(cont)*".jld")["Imaginary"]
# Ȧ₁ = load("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/Box"*string(n)*string(cont)*".jld")["Realvel"]
# Ȧ₂ = load("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/Box"*string(n)*string(cont)*".jld")["RealIm"]

t₁ = 0.4 + 0.5*cont

# low = load("Final/Saving/"*string(poggers)*"/"*string(r)*"/"*string(s)*"/Strings"*string(n)*string(cont)*".jld")["lo"]



repea = 13



#FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repea,cont,low,Ȧ,angll,poggers,cap,pic)

FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,1,6,repea,cont,low,Ȧ,angll,1,cap,pic)                
                
end
        
end

end
        

    

    println(' ')
end

512
5 0.9
1.4
1.9
2.4
2.9
3.4003042357701383
3.9000891918471448
4.4001275772922215
4.900074920859946
5.400082011818742
5.900000846341929
6.400001211404268
6.900028483328479
 
3177.520418 seconds (3.03 M allocations: 67.135 GiB, 0.04% gc time)


In [12]:
@time begin

#         λᵪ = [1024 1448 3072 3584 5504 1];
#     #ηₓ,η = ηtime(time,fₐ);
#     λ = λᵪ[r]

#     ηᵪ = [2.8 2.8 3.2 3.4 3.6 0 400]

    
n = 5
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
i = 4
    
eta = [1 5 7 2 3 4]
rr = [1 5 2 3 4 6]
pog = [1 2 3 4 5 6 6.68]
    
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]





cont = 0
low = 0
Ȧ = 0
angle = 0
cap = 25
Ȧ = 0
angll = 0
pic = 0
#poggers = 6.68

t₀ = 0.0001;
print(string(i)*" ")


Ȧ₁ = zeros(N,N);
Ȧ₂ = zeros(N,N);


μ,σ = 0, 0.1;
C₁ = rand(Normal(μ,σ),N,N);
C₂ = rand(Normal(μ,σ),N,N);


Δx = round(LPQ[7]/N,digits = 4)
Δt = 0.004#0.000001

     repea = 8



#low = load("PQTOWN/new/Saving/"*string(i)*"/Strings"*string(n)*string(repeat)*".jld")["lo"]



# FPQrun_2D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repea,cont,low)


            




LQCD = LPQ/(repea*100)*0.4

Δx = round(LQCD[7]/N,digits = 6)
Δt = 0.001


        #cont = 90
for poggrs ∈ 7:7

for aa ∈ 1:1

for nl ∈ 3:3


repea = 8

C₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box9"*string(n)*string(repea)*".jld")["Real"]
C₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box9"*string(n)*string(repea)*".jld")["Imaginary"]
Ȧ₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box9"*string(n)*string(repea)*".jld")["Realvel"]
Ȧ₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box9"*string(n)*string(repea)*".jld")["RealIm"]

t₁ = 0.4 + 0.5*cont

#         #low = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Strings"*string(n)*string(cont)*".jld")["lo"]

s = eta[nl]
r = rr[aa]

repea = 13

poggers = pog[poggrs]

FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repea,cont,low,Ȧ,angll,poggers,cap,pic)

end
        
end

end
        
        
# for poggrs ∈ 1:1

# for aa ∈ 1:5

# for nl ∈ 1:1


# repea = 8

# C₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Real"]
# C₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Imaginary"]
# Ȧ₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Realvel"]
# Ȧ₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["RealIm"]

# t₁ = 0.4 + 0.5*cont

# #         #low = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Strings"*string(n)*string(cont)*".jld")["lo"]

# s = eta[nl]
# r = rr[aa]

# repea = 13

# poggers = pog[poggrs]

# FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repea,cont,low,Ȧ,angll,poggers,cap,pic)

# end
        
# end

# end
            
   
# for poggrs ∈ 1:1

# for aa ∈ 1:1

# for nl ∈ 1:6


# repea = 8

# C₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Real"]
# C₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Imaginary"]
# Ȧ₁ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["Realvel"]
# Ȧ₂ = load("PQTOWN/new/Saving/"*string(i)*"/Box"*string(n)*string(repea)*".jld")["RealIm"]

# t₁ = 0.4 + 0.5*cont

# #         #low = load("QCDTOWN/rs/Saving/1/1/"*string(i)*"/Strings"*string(n)*string(cont)*".jld")["lo"]

# s = eta[nl]
# r = rr[aa]

# repea = 13

# poggers = pog[poggrs]

# FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repea,cont,low,Ȧ,angll,poggers,cap,pic)

# end
        
# end


# end


    

    println(' ')
end

32
4 0.9
1.4
1.9
2.4
2.9
3.4003042357701383
3.9000891918471448
4.4001275772922215
4.900074920859946
5.400082011818742
5.900000846341929
6.400001211404268
6.900028483328479
 
 23.451949 seconds (2.45 M allocations: 434.592 MiB, 0.42% gc time, 1.35% compilation time)


In [13]:
n = 9

N = 2^n

k_freq = fftfreq(N)*N
kx,ky = meshgrid(k_freq,k_freq)

knrm = sqrt.( kx.^2 + ky.^2)
knrm = collect(Iterators.flatten(knrm))

kbins = range(0.5, N/2+1, step = 1)
kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

1.0:1.0:256.0

In [62]:
@time begin


tracker = load("Final/Saving/1/1/1/Strings913.jld")["time"]
    

# sax111 = load("Final/Saving/1/1/1/FFT913.jld")["saxion"]
# sax211 = load("Final/Saving/2/1/1/FFT913.jld")["saxion"]
# sax311 = load("Final/Saving/3/1/1/FFT913.jld")["saxion"]
# sax411 = load("Final/Saving/4/1/1/FFT913.jld")["saxion"]
# sax511 = load("Final/Saving/5/1/1/FFT913.jld")["saxion"]
# sax611 = load("Final/Saving/6/1/1/FFT913.jld")["saxion"]
# sax66811 = load("Final/Saving/6.68/1/1/FFT913.jld")["saxion"]

# ax111 = load("Final/Saving/1/1/1/FFT913.jld")["axion"]
# ax211 = load("Final/Saving/2/1/1/FFT913.jld")["axion"]
# ax311 = load("Final/Saving/3/1/1/FFT913.jld")["axion"]
# ax411 = load("Final/Saving/4/1/1/FFT913.jld")["axion"]
# ax511 = load("Final/Saving/5/1/1/FFT913.jld")["axion"]
# ax611 = load("Final/Saving/6/1/1/FFT913.jld")["axion"]
# ax66811 = load("Final/Saving/6.68/1/1/FFT913.jld")["axion"]
    
cores111 = load("Final/Saving/1/1/1/Strings913.jld")["number"]
cores211 = load("Final/Saving/2/1/1/Strings913.jld")["number"]
cores311 = load("Final/Saving/3/1/1/Strings913.jld")["number"]
cores411 = load("Final/Saving/4/1/1/Strings913.jld")["number"]
cores511 = load("Final/Saving/5/1/1/Strings913.jld")["number"]
cores611 = load("Final/Saving/6/1/1/Strings913.jld")["number"]
cores66811 = load("Final/Saving/6.68/1/1/Strings913.jld")["number"]




# for i ∈ 1:length(sax66811)

#     plotc = Plots.plot!(kvals,sax111[i],xaxis =:log ,yaxis =:log,legend = true,label = "n = 1" , ylims = (10^3,10^9),
#         title = "512: Changing n, Saxion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")

#     plotc = Plots.plot!(kvals,sax211[i], label = "n = 2")
#     plotc = Plots.plot!(kvals,sax311[i], label = "n = 3")
#     plotc = Plots.plot!(kvals,sax411[i], label = "n = 4")
#     plotc = Plots.plot!(kvals,sax511[i], label = "n = 5")
#     plotc = Plots.plot!(kvals,sax611[i], label = "n = 6")
#     plotc = Plots.plot!(kvals,sax66811[i], label = "n = 6.68")
    


#     Plots.savefig(plotc,"Final/Saving/n/Saxion/"*lpad( string(i-1) ,3,"0")*"nsax"*".png")

#     plotc = Plots.plot()

# end


# for i ∈ 1:length(ax66811)

#     plotc = Plots.plot!(kvals,ax111[i],xaxis =:log ,yaxis =:log,legend = true,label = "n = 1" , ylims = (10^3,10^11),
#         title = "512: Changing n, Axion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")

#     plotc = Plots.plot!(kvals,ax211[i], label = "n = 2")
#     plotc = Plots.plot!(kvals,ax311[i], label = "n = 3")
#     plotc = Plots.plot!(kvals,ax411[i], label = "n = 4")
#     plotc = Plots.plot!(kvals,ax511[i], label = "n = 5")
#     plotc = Plots.plot!(kvals,ax611[i], label = "n = 6")
#     plotc = Plots.plot!(kvals,ax611[i], label = "n = 6.68")


#     Plots.savefig(plotc,"Final/Saving/n/Axion/"*lpad( string(i-1) ,3,"0")*"nax"*".png")

#     plotc = Plots.plot()

# end


plotf = Plots.plot()
plotf = Plots.plot!(tracker,cores111,title = "String Count for Changing n",legend = true,label = "n = 1",xaxis =:log ,yaxis = :log,xlims = (0.4,6.9), ylims = (1,5e3),xlabel = "η",ylabel = "String Number")
plotf = Plots.plot!(tracker,cores211,label = "n = 2")
plotf = Plots.plot!(tracker,cores311,label = "n = 3")
plotf = Plots.plot!(tracker,cores411,label = "n = 4")
plotf = Plots.plot!(tracker,cores511,label = "n = 5")
plotf = Plots.plot!(tracker,cores611,label = "n = 6")
plotf = Plots.plot!(tracker,cores66811,label = "n = 6.68")

Plots.savefig(plotf,"Final/Saving/n/nStrings.png")


end

  1.703939 seconds (638.76 k allocations: 23.375 MiB, 2.47% compilation time)


┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/ebPq2/src/ticks.jl:295
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/ebPq2/src/ticks.jl:295
┌ Warning: Invalid negative or zero value 0.0 found at series index 87 for log10 based yscale
└ @ Plots /home/max/.julia/packages/Plots/CBYO0/src/utils.jl:95


In [10]:
@time begin


tracker = load("Final/Saving/1/1/1/Strings913.jld")["time"]


# sax111 = load("Final/Saving/1/1/1/FFT913.jld")["saxion"]
# sax112 = load("Final/Saving/1/1/2/FFT913.jld")["saxion"]
# sax113 = load("Final/Saving/1/1/3/FFT913.jld")["saxion"]
# sax114 = load("Final/Saving/1/1/4/FFT913.jld")["saxion"]
# sax115 = load("Final/Saving/1/1/5/FFT913.jld")["saxion"]
# sax117 = load("Final/Saving/1/1/7/FFT913.jld")["saxion"]

# ax111 = load("Final/Saving/1/1/1/FFT913.jld")["axion"]
# ax112 = load("Final/Saving/1/1/2/FFT913.jld")["axion"]
# ax113 = load("Final/Saving/1/1/3/FFT913.jld")["axion"]
# ax114 = load("Final/Saving/1/1/4/FFT913.jld")["axion"]
# ax115 = load("Final/Saving/1/1/5/FFT913.jld")["axion"]
# ax117 = load("Final/Saving/1/1/7/FFT913.jld")["axion"]
    
    
cores110 = load("Final/Saving/1/1/6/Strings913.jld")["number"]
cores111 = load("Final/Saving/1/1/1/Strings913.jld")["number"]
cores112 = load("Final/Saving/1/1/2/Strings913.jld")["number"]
cores113 = load("Final/Saving/1/1/3/Strings913.jld")["number"]
cores114 = load("Final/Saving/1/1/4/Strings913.jld")["number"]
cores115 = load("Final/Saving/1/1/5/Strings913.jld")["number"]
cores117 = load("Final/Saving/1/1/7/Strings913.jld")["number"]
    

# for i ∈ 1:length(sax117)

#     plotc = Plots.plot!(kvals,sax111[i],xaxis =:log ,yaxis =:log,legend = true,label = "ηₓ = 2.8" , ylims = (10^3,10^9),
#         title = "512: Changing ηₓ, Saxion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")

#     plotc = Plots.plot!(kvals,sax112[i], label = "ηₓ = 3.0")
#     plotc = Plots.plot!(kvals,sax113[i], label = "ηₓ = 3.2")
#     plotc = Plots.plot!(kvals,sax114[i], label = "ηₓ = 3.4")
#     plotc = Plots.plot!(kvals,sax115[i], label = "ηₓ = 3.6")
#     plotc = Plots.plot!(kvals,sax117[i], label = "ηₓ = 6.0")

#     Plots.savefig(plotc,"Final/Saving/eta/Saxion/"*lpad( string(i-1) ,3,"0")*"etasax"*".png")

#     plotc = Plots.plot()

# end


# for i ∈ 1:length(ax117)

#     plotc = Plots.plot!(kvals,ax111[i],xaxis =:log ,yaxis =:log,legend = true,label = "ηₓ = 2.8" , ylims = (10^3,10^11),
#         title = "512: Changing ηₓ, Axion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")

#     plotc = Plots.plot!(kvals,ax112[i], label = "ηₓ = 3.0")
#     plotc = Plots.plot!(kvals,ax113[i], label = "ηₓ = 3.2")
#     plotc = Plots.plot!(kvals,ax114[i], label = "ηₓ = 3.4")
#     plotc = Plots.plot!(kvals,ax115[i], label = "ηₓ = 3.6")
#     plotc = Plots.plot!(kvals,ax117[i], label = "ηₓ = 6.0")


#     Plots.savefig(plotc,"Final/Saving/eta/Axion/"*lpad( string(i-1) ,3,"0")*"etaax"*".png")

#     plotc = Plots.plot()

# end
    

plotf = Plots.plot()
plotf = Plots.plot!(tracker,cores110,title = "String Count for Changing ηₓ",legend = true,label = "ηₓ = 0",xaxis =:log ,yaxis = :log,xlims = (0.4,6.9),ylims = (1,5e3),xlabel = "η",ylabel = "String Number")
plotf = Plots.plot!(tracker,cores111,label = "ηₓ = 2.8")
plotf = Plots.plot!(tracker,cores112,label = "ηₓ = 3.0")
plotf = Plots.plot!(tracker,cores113,label = "ηₓ = 3.2")
plotf = Plots.plot!(tracker,cores114,label = "ηₓ = 3.4")
plotf = Plots.plot!(tracker,cores115,label = "ηₓ = 3.6")
plotf = Plots.plot!(tracker,cores117,label = "ηₓ = 6.0")

Plots.savefig(plotf,"Final/Saving/eta/etaStrings.png")


end

┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/ebPq2/src/ticks.jl:295
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/ebPq2/src/ticks.jl:295
┌ Warning: Invalid negative or zero value 0.0 found at series index 138 for log10 based yscale
└ @ Plots /home/max/.julia/packages/Plots/CBYO0/src/utils.jl:95


 26.021314 seconds (51.59 M allocations: 2.900 GiB, 6.01% gc time, 29.89% compilation time)


In [64]:
@time begin


tracker = load("Final/Saving/1/1/1/Strings913.jld")["time"]

    
# sax111 = load("Final/Saving/1/1/1/FFT913.jld")["saxion"]
# sax121 = load("Final/Saving/1/2/1/FFT913.jld")["saxion"]
# sax131 = load("Final/Saving/1/3/1/FFT913.jld")["saxion"]
# sax141 = load("Final/Saving/1/4/1/FFT913.jld")["saxion"]
# sax151 = load("Final/Saving/1/5/1/FFT913.jld")["saxion"]

# ax111 = load("Final/Saving/1/1/1/FFT913.jld")["axion"]
# ax121 = load("Final/Saving/1/2/1/FFT913.jld")["axion"]
# ax131 = load("Final/Saving/1/3/1/FFT913.jld")["axion"]
# ax141 = load("Final/Saving/1/4/1/FFT913.jld")["axion"]
# ax151 = load("Final/Saving/1/5/1/FFT913.jld")["axion"]

cores111 = load("Final/Saving/1/1/1/Strings913.jld")["number"]
cores121 = load("Final/Saving/1/2/1/Strings913.jld")["number"]
cores131 = load("Final/Saving/1/3/1/Strings913.jld")["number"]
cores141 = load("Final/Saving/1/4/1/Strings913.jld")["number"]
cores151 = load("Final/Saving/1/5/1/Strings913.jld")["number"]
    

# for i ∈ 1:length(sax151)

#     plotc = Plots.plot!(kvals,sax111[i],xaxis =:log ,yaxis =:log,legend = true,label = "λ = 1024" , ylims = (10^3,10^9),
#         title = "512: Changing λ, Saxion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")

#     plotc = Plots.plot!(kvals,sax121[i], label = "λ = 1448")
#     plotc = Plots.plot!(kvals,sax131[i], label = "λ = 3072")
#     plotc = Plots.plot!(kvals,sax141[i], label = "λ = 3584")
#     plotc = Plots.plot!(kvals,sax151[i], label = "λ = 5504")


#     Plots.savefig(plotc,"Final/Saving/lambda/Saxion/"*lpad( string(i-1) ,3,"0")*"lamsax"*".png")

#     plotc = Plots.plot()

# end


# for i ∈ 1:length(ax151)

#     plotc = Plots.plot!(kvals,ax111[i],xaxis =:log ,yaxis =:log,legend = true,label = "λ = 1024" , ylims = (10^3,10^11),
#         title = "512: Changing λ, Axion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")

#     plotc = Plots.plot!(kvals,ax121[i], label = "λ = 1448")
#     plotc = Plots.plot!(kvals,ax131[i], label = "λ = 3072")
#     plotc = Plots.plot!(kvals,ax141[i], label = "λ = 3584")
#     plotc = Plots.plot!(kvals,ax151[i], label = "λ = 5504")


#     Plots.savefig(plotc,"Final/Saving/lambda/Axion/"*lpad( string(i-1) ,3,"0")*"lamax"*".png")
    
#     plotc = Plots.plot()

# end
    

plotf = Plots.plot()
plotf = Plots.plot!(tracker,cores111,title = "String Count for Changing λ",legend = true,label = "λ = 1024",xaxis =:log ,yaxis = :log,xlims = (0.4,6.9),ylims = (1,5e3),xlabel = "η",ylabel = "String Number")
plotf = Plots.plot!(tracker,cores121,label = "λ = 1448")
plotf = Plots.plot!(tracker,cores131,label = "λ = 3072")
plotf = Plots.plot!(tracker,cores141,label = "λ = 3584")
plotf = Plots.plot!(tracker,cores151,label = "λ = 5504")

Plots.savefig(plotf,"Final/Saving/lambda/lamStrings.png")


end

  1.287466 seconds (424.42 k allocations: 10.872 MiB)


┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/ebPq2/src/ticks.jl:295
┌ Warning: No strict ticks found
└ @ PlotUtils /home/max/.julia/packages/PlotUtils/ebPq2/src/ticks.jl:295
┌ Warning: Invalid negative or zero value 0.0 found at series index 87 for log10 based yscale
└ @ Plots /home/max/.julia/packages/Plots/CBYO0/src/utils.jl:95


In [68]:
@time begin


tracker = load("PQTOWN/new/Saving/5/Strings98.jld")["time"]
sax = load("PQTOWN/new/Saving/5/FFT98.jld")["saxion"]
ax = load("PQTOWN/new/Saving/5/FFT98.jld")["axion"]

cores = load("PQTOWN/new/Saving/5/Strings98.jld")["number"]
print(cores[1])
    plotc = Plots.plot()

for i ∈ 1:length(sax)

    plotc = Plots.plot!(kvals,sax[i],xaxis =:log ,yaxis =:log,legend = false , ylims = (10^0,10^7),
        title = "512: PQ Epoch, Saxion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")


    Plots.savefig(plotc,"Final/Saving/PQ/Saxion/"*lpad( string(i-1) ,3,"0")*"pqsax"*".png")

    plotc = Plots.plot()

end

for i ∈ 1:length(ax)

    plotc = Plots.plot!(kvals,ax[i],xaxis =:log ,yaxis =:log,legend = false , ylims = (10^6,10^11),
        title = "512: PQ Epoch, Axion η = "*string(round(tracker[i],digits = 3)),xlabel = "Wavenumber",ylabel = "P")


    Plots.savefig(plotc,"Final/Saving/PQ/Axion/"*lpad( string(i-1) ,3,"0")*"pqax"*".png")

    plotc = Plots.plot()

end

    
# plotf = Plots.plot()
# plotf = Plots.plot!(tracker,cores,title = "String Count for PQ Epoch",legend = false ,xaxis =:log,yaxis = :log,xlims = (1e0,8e2),ylims = (2.8e2,1e5),xlabel = "η",ylabel = "String Number")

# Plots.savefig(plotf,"Final/Saving/PQ/PQStrings.png")
    

end

30126 34.030343 seconds (251.93 M allocations: 4.670 GiB, 1.94% gc time)
